# Object detector dataset generator

With this notebook you'll be able to artificially generate and automatically label
a dataset for detecting objects. You can bring your own images or test it by fetching images from Kaggle hub.


If running on colab, install dependencies:


In [ ]:
!pip install numpy opencv-python pillow pycocotools pyyaml torch ultralytics matplotlib imutils argparse groundingdino-py segment-anything

If you want to mount your google drive for training with checkpoints in colab and saving your progress run the following cell:


In [ ]:
!rm -rf /content/sample_data
!rm -rf /content/.config
from google.colab import drive
import os
drive.mount('/content', force_remount=True)
workdir = "/content/MyDrive/RoBorregos/vision/dataset_generator/"
os.makedirs(workdir, exist_ok=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

If not mounting a drive run the next cell:


In [ ]:
workdir = "/home/roborregos/visao/home-pipelines/vision/object_detector/content/"

In [ ]:
from groundingdino.util.vl_utils import create_positive_map_from_span
from groundingdino.util.utils import clean_state_dict, get_phrases_from_posmap
from groundingdino.util.slconfig import SLConfig
from groundingdino.util.inference import load_model, predict
from groundingdino.util import box_ops
from groundingdino.models import build_model
import groundingdino.datasets.transforms as T
from segment_anything import sam_model_registry, SamPredictor
import argparse
import imutils
import time
import ultralytics
from pathlib import Path
import matplotlib.pyplot as plt
import torch
import csv
import yaml
from pycocotools import mask
from PIL import Image, ImageDraw, ImageEnhance, ImageFilter, ImageFont, UnidentifiedImageError
import cv2
import numpy as np
import json
import random
import sys
import os
os.chdir("/")

# Attention!

If you have specific pictures, place the folders in ./images and skip the next two blocks of code which download a default dataset.


In [ ]:
!pip install kagglehub
import kagglehub

dataset_path = kagglehub.dataset_download("bhavikjikadara/dog-and-cat-classification-dataset")

In [ ]:
!cp -r {dataset_path}/PetImages {workdir}
!mv {workdir}/PetImages {workdir}/images

def clean_and_trim_dataset(folder):
    total_removed = 0

    for category in ["Cat", "Dog"]:
        path = os.path.join(folder, category)
        valid_images = []

        # Step 1: Remove corrupt images
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            try:
                with Image.open(img_path) as img:
                    img.verify()
                valid_images.append(img_path)
            except (UnidentifiedImageError, OSError):
                os.remove(img_path)
                total_removed += 1

        print(f"Removed {total_removed} corrupt images from {category}")

        # Step 2: Remove part of the remaining valid images
        to_delete = random.sample(valid_images, len(valid_images) * 99 // 100)
        for img_path in to_delete:
            os.remove(img_path)

        print(f"Removed {len(to_delete)} images from {category} to reduce dataset size")

clean_and_trim_dataset(workdir + "images")

def count_files_in_dir(directory):
    # List all files (images) in the directory and count them
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    return len(files)

# Example usage for "Cat" and "Dog" folders
cat_folder = workdir + "images/Cat"
dog_folder = workdir + "images/Dog"

print(f"Number of cat images: {count_files_in_dir(cat_folder)}")
print(f"Number of dog images: {count_files_in_dir(dog_folder)}")

## Crop images to 1:1


In [ ]:
from PIL import Image, UnidentifiedImageError, ImageOps
import os

base_path = workdir + "images"

for class_dir in os.listdir(base_path):
    class_path = os.path.join(base_path, class_dir)

    if not os.path.isdir(class_path):
        continue
    print(class_path)
    for filename in os.listdir(class_path):
        file_path = os.path.join(class_path, filename)

        try:
            with Image.open(file_path) as img:
                # Apply EXIF orientation
                img = ImageOps.exif_transpose(img)

                width, height = img.size
                min_dim = min(width, height)

                left = (width - min_dim) // 2
                top = (height - min_dim) // 2
                right = left + min_dim
                bottom = top + min_dim

                img_cropped = img.crop((left, top, right, bottom))
                img_cropped.save(file_path)
        except (UnidentifiedImageError, OSError) as e:
            print(f"Removing corrupt file: {file_path}")
            os.remove(file_path)

os.chdir(workdir)

## Resize images

If tight on time, consider resizing down the images for faster segmentation and other processes, if not skip this step for better dataset quality (considering the images are high resolution).


In [ ]:
base_path = workdir + "images"
size = 1280

for class_dir in os.listdir(base_path):
    class_path = os.path.join(base_path, class_dir)

    if not os.path.isdir(class_path):
        continue
    print(class_path)
    for filename in os.listdir(class_path):
        file_path = os.path.join(class_path, filename)

        try:
            with Image.open(file_path) as img:
                img = img.resize((size, size))
                img.save(file_path)
        except (UnidentifiedImageError, OSError) as e:
            print(f"Removing corrupt file: {file_path}")
            os.remove(file_path)  # delete corrupt image

os.chdir(workdir)

## Rotate images

If needed rotate the image an amount of degrees.


In [ ]:
import os
from PIL import Image

angle = 90


def rotate_images_in_directory(directory):
    count = 0
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            try:
                with Image.open(filepath) as img:
                    rotated = img.rotate(angle, expand=True)
                    rotated.save(filepath)
                    count += 1
            except Exception as e:
                print(f"Failed to process {filename}: {e}")
    print(f"Rotated {count} images in {directory}")


def rotate_all_subdirs(base_directory):
    for subdir in os.listdir(base_directory):
        subdir_path = os.path.join(base_directory, subdir)
        if os.path.isdir(subdir_path):
            rotate_images_in_directory(subdir_path)


# Example usage
# rotate_all_subdirs(os.path.join(workdir, "processed/Apple"))
rotate_images_in_directory(workdir + "processed/Soap")

## Download models

If you have already ran this and are using a volume/drive there is no need to run again.


In [ ]:
!git clone https://github.com/IDEA-Research/GroundingDINO.git
!wget https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

## Segment images


In [ ]:
SAVE_BB = False
DEBUG = False

# path to save results already processed and segmented images
results_path = workdir + "processed"
# change the path of the model config file
config_file = workdir + "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py"
# change the path of the model
checkpoint_path = workdir + "groundingdino_swint_ogc.pth"
output_dir = results_path
box_threshold = 0.3
text_threshold = 0.25
token_spans = None

sys.path.append("..")
sam_model = "h"

# use sam model
# wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
# wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_l_0b3195.pth
if sam_model == "h":
    sam_checkpoint = workdir + "sam_vit_h_4b8939.pth"
    model_type = "vit_h"
else:
    sam_checkpoint = workdir + "sam_vit_l_0b3195.pth"
    model_type = "vit_l"

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using", device)
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
predictor = SamPredictor(sam)

images = []
annotations = []
categories = []

img_id = 0
anno_id = 0

# Make a list of all the directories in the path
base_path = workdir + "images"
path_to_classes = [f.path for f in os.scandir(base_path) if f.is_dir()]


def load_image(image_path):

    image_pil = Image.open(image_path).convert("RGB")  # load image

    transform = T.Compose(
        [
            T.RandomResize([800], max_size=1333),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    )
    image, _ = transform(image_pil, None)  # 3, h, w
    return image_pil, image


def load_model(model_config_path, model_checkpoint_path, cpu_only=False):
    args = SLConfig.fromfile(model_config_path)
    args.device = device
    model = build_model(args)
    checkpoint = torch.load(model_checkpoint_path, map_location="cpu")
    load_res = model.load_state_dict(
        clean_state_dict(checkpoint["model"]), strict=False)
    if DEBUG:
        print(load_res)
    _ = model.eval()
    return model


def get_grounding_output(model, image, caption, box_threshold, text_threshold=None, with_logits=True, cpu_only=False, token_spans=None):
    assert text_threshold is not None or token_spans is not None, "text_threshould and token_spans should not be None at the same time!"
    caption = caption.lower()
    caption = caption.strip()
    if not caption.endswith("."):
        caption = caption + "."
    model = model.to(device)
    image = image.to(device)
    with torch.no_grad():
        if DEBUG:
            print("Running model...")
        outputs = model(image[None], captions=[caption])
    logits = outputs["pred_logits"].sigmoid()[0]  # (nq, 256)
    boxes = outputs["pred_boxes"][0]  # (nq, 4)

    # filter output
    if token_spans is None:
        logits_filt = logits.cpu().clone()
        boxes_filt = boxes.cpu().clone()
        filt_mask = logits_filt.max(dim=1)[0] > box_threshold
        logits_filt = logits_filt[filt_mask]  # num_filt, 256
        boxes_filt = boxes_filt[filt_mask]  # num_filt, 4

        # get phrase
        tokenlizer = model.tokenizer
        tokenized = tokenlizer(caption)
        # build pred
        pred_phrases = []
        for logit, box in zip(logits_filt, boxes_filt):
            pred_phrase = get_phrases_from_posmap(
                logit > text_threshold, tokenized, tokenlizer)
            if with_logits:
                pred_phrases.append(
                    pred_phrase + f"({str(logit.max().item())[:4]})")
            else:
                pred_phrases.append(pred_phrase)
    else:
        # given-phrase mode
        positive_maps = create_positive_map_from_span(
            model.tokenizer(text_prompt),
            token_span=token_spans
        ).to(image.device)  # n_phrase, 256

        logits_for_phrases = positive_maps @ logits.T  # n_phrase, nq
        all_logits = []
        all_phrases = []
        all_boxes = []
        for (token_span, logit_phr) in zip(token_spans, logits_for_phrases):
            # get phrase
            phrase = ' '.join([caption[_s:_e] for (_s, _e) in token_span])
            # get mask
            filt_mask = logit_phr > box_threshold
            # filt box
            all_boxes.append(boxes[filt_mask])
            # filt logits
            all_logits.append(logit_phr[filt_mask])
            if with_logits:
                logit_phr_num = logit_phr[filt_mask]
                all_phrases.extend(
                    [phrase + f"({str(logit.item())[:4]})" for logit in logit_phr_num])
            else:
                all_phrases.extend([phrase for _ in range(len(filt_mask))])
        boxes_filt = torch.cat(all_boxes, dim=0).cpu()
        pred_phrases = all_phrases
    return boxes_filt, pred_phrases


def verify_or_create_dir(path):
    os.makedirs(path, exist_ok=True)
    if DEBUG:
        print(f"Verified/created: {path}")


def count_all_files_in_dir(directory):
    count = 0
    for root, dirs, files in os.walk(directory):
        count += len([f for f in files if os.path.isfile(os.path.join(root, f))])
    return count


# Get total number of images to process
image_dir = workdir + "images"
number_of_images = count_all_files_in_dir(image_dir)
print(f"Total image files: {number_of_images}")

# Check if results directory exists, else create it
verify_or_create_dir(results_path)

# Main loop
i = 0
for class_path in path_to_classes:
    imgPaths = os.listdir(class_path)
    if SAVE_BB:
        class_name = os.path.basename(class_path)
        verify_or_create_dir(f"{results_path}/bbs/{class_name}")

    for imgPath in imgPaths:
        if DEBUG:
            print(f"Processing image: {imgPath}")
        print(f"%{i * 100 / number_of_images}")
        img = imutils.resize(cv2.imread(f"{class_path}/{imgPath}"))
        if img is None:
            continue

    # ------------------------start grounding----------------------------------------------

        # Image_path = args.image_path
        cpu_only = False if torch.cuda.is_available() else True

        # Load image
        image_pil, image = load_image(f"{class_path}/{imgPath}")

        # Load model
        model = load_model(config_file, checkpoint_path, cpu_only=cpu_only)

        # Set the text_threshold to None if token_spans is set.
        if token_spans is not None:
            text_threshold = None
            print("Using token_spans. Set the text_threshold to None.")

        # Run model
        text_prompt = os.path.basename(class_path)
        boxes_filt, pred_phrases = get_grounding_output(
            model, image, text_prompt, box_threshold, text_threshold, cpu_only=cpu_only, token_spans=eval(
                f"{token_spans}")
        )

        # Found bb dimensions

        size = image_pil.size
        pred_dict = {
            "boxes": boxes_filt,
            "size": [size[1], size[0]],  # H, W
            "labels": pred_phrases,
        }

        H, W = pred_dict["size"]
        boxes = pred_dict["boxes"]
        labels = pred_dict["labels"]
        assert len(boxes) == len(
            labels), "boxes and labels must have same length"

        draw = ImageDraw.Draw(image_pil)
        mask = Image.new("L", image_pil.size, 0)
        mask_draw = ImageDraw.Draw(mask)

        # change pil image to cv2 image
        img = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)
        img2 = img.copy()
        # draw boxes and masks
        for box, label in zip(boxes, labels):
            # from 0..1 to 0..W, 0..H
            box = box * torch.Tensor([W, H, W, H])
            # from xywh to xyxy
            box[:2] -= box[2:] / 2
            box[2:] += box[:2]
            # random color
            color = tuple(np.random.randint(0, 255, size=1).tolist())
            # draw
            padding = 10
            x0, y0, x1, y1 = box
            x0, y0, x1, y1 = int(x0)-padding, int(y0) - \
                padding, int(x1)+padding, int(y1)+padding

            # validate if the bounding box is inside the image
            if x0 < 0:
                x0 = 0
            if y0 < 0:
                y0 = 0
            if x1 > W:
                x1 = W
            if y1 > H:
                y1 = H

            # draw rectangles
            cv2.rectangle(img2, (x0, y0), (x1, y1), color, 2)

            draw.rectangle([x0, y0, x1, y1], outline=color, width=6)
            # draw.text((x0, y0), str(label), fill=color)

            font = ImageFont.load_default()
            if hasattr(font, "getbbox"):
                bbox = draw.textbbox((x0, y0), str(label), font)
            else:
                w, h = draw.textsize(str(label), font)
                bbox = (x0, y0, w + x0, y0 + h)
            # bbox = draw.textbbox((x0, y0), str(label))
            draw.rectangle(bbox, fill=color)
            draw.text((x0, y0), str(label), fill="white")

            mask_draw.rectangle([x0, y0, x1, y1], fill=255, width=6)

    # ----------------Start SAM--------------------------------------------------------------

            class_name = class_path.split("/")[-1]
            sam_bounding_box = np.array([x0, y0, x1, y1])
            ran_sam = False
            # run sam
            if ran_sam == False:
                predictor.set_image(img)
                ran_sam = True

            mask, _, _ = predictor.predict(
                point_coords=None,
                point_labels=None,
                box=sam_bounding_box,
                multimask_output=False,
            )

            mask, _, _ = predictor.predict(
                box=sam_bounding_box, multimask_output=False)

            # Make png mask
            contours, _ = cv2.findContours(mask[0].astype(
                np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # Your call to find the contours

            # threshold input image using otsu thresholding as mask and refine with morphology
            ret, pngmask = cv2.threshold(mask[0].astype(
                np.uint8), 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            kernel = np.ones((9, 9), np.uint8)
            pngmask = cv2.morphologyEx(pngmask, cv2.MORPH_CLOSE, kernel)
            pngmask = cv2.morphologyEx(pngmask, cv2.MORPH_OPEN, kernel)
            result = img.copy()
            result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
            result[:, :, 3] = pngmask

    # ----------------Save Images-----------------------------------------------------------------

            if SAVE_BB:
                cv2.imwrite(f"{results_path}/bbs/{class_name}/{imgPath}", img2)

            verify_or_create_dir(f"{results_path}/{class_name}")

            file_path = f"{results_path}/{class_name}/{imgPath[:-4]}.png"
            if os.path.exists(file_path):
                if os.path.exists(f"{results_path}/{class_name}/{imgPath[:-4]}_1.png"):
                    if DEBUG:
                        print("File already exists, saving with _2")
                    cv2.imwrite(
                        f"{results_path}/{class_name}/{imgPath[:-4]}_2.png", result)
                else:
                    if DEBUG:
                        print("File already exists, saving with _1")
                    file_path = f"{results_path}/{class_name}/{imgPath[:-4]}_1.png"

            cv2.imwrite(file_path, result)
            ran_sam = False
        i = i + 1

## Crop processed images


In [ ]:
from PIL import Image


def verify_or_create_dir(path):
    os.makedirs(path, exist_ok=True)


results_path = workdir + "DS_res/"
path_to_classes = [f.path for f in os.scandir(
    workdir + "processed") if f.is_dir()]

for class_path in path_to_classes:
    class_name = os.path.basename(class_path)
    verify_or_create_dir(results_path + class_name)
    for file_name in os.listdir(class_path):
        try:
            file_path = class_path + "/" + file_name
            my_image = Image.open(file_path)
            black = Image.new('RGBA', my_image.size)
            my_image = Image.composite(my_image, black, my_image)
            cropped_image = my_image.crop(my_image.getbbox())
            cropped_image.save(f"{results_path}{class_name}/{file_name}")
            print(f"{file_name} done")
        except Exception as e:
            print(f"{file_name} failed {e}")
            continue

## Manually check segmented images

Press k to keep an images or d to delete it. You may also use the buttons.


In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from PIL import Image as PILImage
import io
import base64

# --- CONFIG ---
image_dir = workdir + 'DS_res'  # your directory
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff', '.webp')
batch_size = 12
grid_cols = 4
thumb_size = (200, 200)

# --- Collect images ---
image_paths = []
for root, _, files in os.walk(image_dir):
    for file in files:
        if file.lower().endswith(image_extensions):
            full_path = os.path.join(root, file)
            if os.path.isfile(full_path):
                image_paths.append(full_path)

print(f"Found {len(image_paths)} images")

# --- State ---
index = {"i": 0}
delete_list = []
output = widgets.Output()
status = widgets.Label()
next_button = widgets.Button(description="Next Batch", button_style='primary')
delete_button = widgets.Button(
    description="Delete Selected", button_style='danger')
confirm_delete = widgets.Button(
    description="Confirm Deletion", button_style='danger')


def show_batch():
    output.clear_output(wait=True)

    start = index["i"]
    end = min(start + batch_size, len(image_paths))
    batch = image_paths[start:end]

    if not batch:
        with output:
            print("✅ Done reviewing all images.")
            if delete_list:
                print(
                    f"🗑️ {len(delete_list)} images marked for deletion. Click 'Confirm Deletion' to delete.")
            display(confirm_delete)
        return

    # Create all widgets for the batch
    image_checkboxes = []
    current_batch_cbs = []  # To store checkboxes for this batch

    for img_path in batch:
        try:
            # Create the checkbox
            cb = widgets.Checkbox(
                description=f"{os.path.basename(img_path)}",
                indent=False,
                layout=widgets.Layout(width='auto')
            )
            cb.image_path = img_path
            current_batch_cbs.append(cb)

            # Load and resize the image
            img = PILImage.open(img_path)
            img.thumbnail(thumb_size)
            buf = io.BytesIO()
            img.save(buf, format='PNG')
            buf.seek(0)
            img_data = buf.getvalue()

            # Create image widget
            img_widget = widgets.Image(
                value=img_data,
                format='png',
                width=200,
                height=200,
                layout=widgets.Layout(
                    margin='0px'
                )
            )

            # Simple VBox container for image and checkbox
            container = widgets.VBox([
                img_widget,
                cb
            ], layout=widgets.Layout(
                border='1px solid #ddd',
                margin='5px',
                padding='5px',
                align_items='center'
            ))

            image_checkboxes.append(container)

        except Exception as e:
            print(f"Error loading {img_path}: {str(e)}")
            # Create an error placeholder with checkbox
            error_widget = widgets.HTML(
                value=f"⚠️ Error loading:<br>{os.path.basename(img_path)}",
                layout=widgets.Layout(
                    height='200px',
                    width='200px',
                    display='flex',
                    align_items='center',
                    justify_content='center'
                )
            )

            cb = widgets.Checkbox(
                description=f"{os.path.basename(img_path)}",
                indent=False,
                layout=widgets.Layout(width='auto')
            )
            cb.image_path = img_path
            current_batch_cbs.append(cb)

            container = widgets.VBox([
                error_widget,
                cb
            ], layout=widgets.Layout(
                border='1px solid #ddd',
                margin='5px',
                padding='5px',
                align_items='center'
            ))

            image_checkboxes.append(container)

    # Store the checkboxes for this batch
    output.current_batch_checkboxes = current_batch_cbs

    # Rest of your show_batch function remains the same...
    # Create grid layout
    grid = []
    for i in range(0, len(image_checkboxes), grid_cols):
        row = image_checkboxes[i:i+grid_cols]
        grid.append(widgets.HBox(row))

    # Add instructions for the user
    instructions = widgets.HTML(
        value="""
        <div style="padding: 10px; background-color: #e3f2fd; border-radius: 5px; margin-bottom: 15px;">
            <p><strong>Instructions:</strong> Review the images and select the checkbox below each image you want to delete.
            Click "Delete Selected" to mark them for deletion and move to the next batch.
            When finished, click "Confirm Deletion" to permanently delete all marked images.</p>
        </div>
        """
    )

    with output:
        display(instructions)

        # Display the grid
        for row in grid:
            display(row)

        # Display buttons
        button_box = widgets.HBox([next_button, delete_button, confirm_delete])
        display(button_box)
        display(status)


def on_next_click(_):
    index["i"] += batch_size
    show_batch()


def on_delete_click(_):
    # We need to track selected checkboxes differently since the widgets are cleared
    # Let's modify show_batch to store the current batch checkboxes
    if hasattr(output, 'current_batch_checkboxes'):
        selected = [
            cb.image_path for cb in output.current_batch_checkboxes if cb.value]
        delete_list.extend(selected)
        status.value = f"🗑️ Marked {len(selected)} new image(s), {len(delete_list)} total for deletion."

    # Move to next batch
    index["i"] += batch_size
    show_batch()


def delete_images(_):
    if not delete_list:
        status.value = "No images selected for deletion."
        return

    deleted = 0
    failed = 0

    for path in delete_list:
        try:
            os.remove(path)
            deleted += 1
        except Exception as e:
            print(f"Failed to delete {path}: {e}")
            failed += 1

    status.value = f"✅ Successfully deleted {deleted} images. {failed} failed."
    delete_list.clear()

    # Refresh the image list
    image_paths.clear()
    for root, _, files in os.walk(image_dir):
        for file in files:
            if file.lower().endswith(image_extensions):
                full_path = os.path.join(root, file)
                if os.path.isfile(full_path):
                    image_paths.append(full_path)

    index["i"] = 0
    show_batch()


next_button.on_click(on_next_click)
delete_button.on_click(on_delete_click)
confirm_delete.on_click(delete_images)

display(output)
show_batch()

## Setup for dataset generation


In [ ]:
from PIL import Image
import os


def verify_or_create_dir(path):
    os.makedirs(path, exist_ok=True)


results_path = workdir + "DS_res/"

# Get all subfolders inside results_path
path_to_classes = [f.path for f in os.scandir(
    results_path) if f.is_dir() and "bbs" not in f.name]

# Build list of (path, class_name) tuples
fg_folders = [(path, os.path.basename(path)) for path in path_to_classes]

# Define folders
bg_folder = workdir + "bg/"
verify_or_create_dir(bg_folder)
output_folder = workdir + "ds_final/"
objects_list = [os.path.basename(class_path) for class_path in path_to_classes]

# If you have a list of original classes, uncomment and fill it
original_classes = [
    #     "exampleClass1", "exampleClass2", "exampleClass3", "exampleClass4",
]

# Add new classes at the end only
all_detected_classes = [os.path.basename(
    class_path) for class_path in path_to_classes]

# Append only the new classes
final_classes = original_classes.copy()
for cls in all_detected_classes:
    if cls not in final_classes:
        final_classes.append(cls)

# Create annotations_ID and categories using final_classes
annotations_ID = {cls: i for i, cls in enumerate(final_classes)}
categories = [{"id": i, "name": cls} for i, cls in enumerate(final_classes)]

print("annotations_ID:", annotations_ID)
print("categories:", categories)

# Load the list of files in each of the folders
fg_files = {}
for folder, category in fg_folders:
    fg_files[category] = os.listdir(folder)

# Define the folder structure
subfolders = [
    "train/images",
    "train/labels",
    "test/images",
    "test/labels",
    "valid/images",
    "valid/labels",
]

# Create them
for sub in subfolders:
    verify_or_create_dir(os.path.join(output_folder, sub))

## Get backgrounds

For this step, add background images to ./bg

You can run the next cell to download and format a default dataset.


In [ ]:
!pip install kagglehub
import kagglehub

# Download latest version
dataset_path = kagglehub.dataset_download("balraj98/stanford-background-dataset")
os.system(f'cp "{dataset_path}/images/"* {workdir}bg/')

dataset_path = kagglehub.dataset_download("adikurniawan/color-dataset-for-color-recognition")
os.system(f'find "{dataset_path}/training_dataset/" -type f -exec cp {{}} {workdir}bg/ \;')

## Remove corrupt images

You con also remove a percentage of images by modifying _delete_percentage_


In [ ]:
delete_percentage = 0  # e.g., 0.1 for 10%
debug = False  # Set to True to print deleted files
min_width = 0  # Minimum width
min_height = 0  # Minimum height


def clean_and_trim_dataset(folder):
    total_removed = 0
    low_res_removed = 0
    valid_images = []

    # Step 1: Remove corrupt images and low resolution ones
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        if not os.path.isfile(img_path):
            continue
        try:
            with Image.open(img_path) as img:
                img.verify()

            with Image.open(img_path) as img:  # reopen to get size
                width, height = img.size
                if width < min_width or height < min_height:
                    if debug:
                        print(
                            f"Deleting low resolution image: {img_path} ({width}x{height})")
                    os.remove(img_path)
                    low_res_removed += 1
                else:
                    valid_images.append(img_path)

        except (UnidentifiedImageError, OSError):
            if debug:
                print(f"Deleting corrupt image: {img_path}")
            os.remove(img_path)
            total_removed += 1

    print(f"Removed {total_removed} corrupt images")
    print(f"Removed {low_res_removed} low resolution images")

    # Step 2: Remove a percentage of the valid images
    to_delete_count = int(len(valid_images) * delete_percentage)
    to_delete = random.sample(valid_images, to_delete_count)
    for img_path in to_delete:
        if debug:
            print(f"Deleting random image for dataset reduction: {img_path}")
        os.remove(img_path)

    print(f"Removed {len(to_delete)} images to reduce dataset size")


def count_files_in_dir(directory):
    files = [f for f in os.listdir(directory) if os.path.isfile(
        os.path.join(directory, f))]
    return len(files)


# Apply on /content/bg
clean_and_trim_dataset(workdir + "bg")

# Count remaining images
print(f"Remaining number of images: {count_files_in_dir(workdir + 'bg')}")

## Resize bg images

Backgrounds don't have to be the best quality, even when scaled up, but they're the canvas for pasting the object images so a good resolution will improve the dataset.


In [ ]:
from PIL import Image
base_path = workdir + "bg"
size = 640
for filename in os.listdir(base_path):
    file_path = os.path.join(base_path, filename)

    try:
        with Image.open(file_path) as img:
            img = img.resize((size, size))
            img.save(file_path)
    except (UnidentifiedImageError, OSError) as e:
        print(f"Removing corrupt file: {file_path}")
        os.remove(file_path)  # delete corrupt image

os.chdir(workdir)

## Image generator

This cell takes the segmented images of the classes and backgrounds and generate images with a random amount of objects in them scattered around.


In [ ]:
# Restart variables
workdir = "/content/drive/MyDrive/RoBorregos/vision/dataset_generator/"
output_folder = workdir + "ds_final/"

In [ ]:
import random
from PIL import Image, ImageDraw, ImageEnhance, ImageFilter, ImageFont, UnidentifiedImageError
import numpy as np
import yaml

images = []
annotations = []
annotations2 = []
annot_csv = []

img_id = int(0)
anno_id = int(0)

min_size_ratio = 0.05  # Objects must be at least 10% of bg size
max_size_ratio = 0.35  # Objects can be at most 75% of bg size

# Define the maximum overlap as a percentage
max_overlap_pct = 25

trainfolder = output_folder + "train/"
validfolder = output_folder + "valid/"

images_to_generate = 50  # change to 5000
max_objects_per_image = 8

for j in range(images_to_generate):

    with open(f'{trainfolder}labels/{img_id}.txt', 'w') as file:
        pass

    # Decide if this image should be empty
    if random.random() < 0.10:  # 10% chance
        num_objects = 0
    else:
        num_objects = random.randint(1, max_objects_per_image)

    fg_categories = random.choices(objects_list, k=num_objects)

    fg_files_selected = [[category, random.choice(
        fg_files[category])] for category in fg_categories]

    fg_imgs = []
    for img in fg_files_selected:
        folder = [f[0] for f in fg_folders if f[1] == img[0]][0]
        fg_img = Image.open(folder + "/" + img[1]).convert("RGBA")
        fg_imgs.append(
            [img[0], Image.open(folder + "/" + img[1]), folder + img[1]])

    bg_files = os.listdir(bg_folder)
    bg_file = random.choice(bg_files)
    bg_img = Image.open(bg_folder + bg_file)
    bg_img = bg_img.convert("RGBA")

    occupied_mask = np.zeros((bg_img.height, bg_img.width), dtype=np.uint8)

    for img in fg_imgs:
        fg_img = img[1]

        angle = random.randint(-5, 5)
        fg_img = fg_img.rotate(angle, resample=Image.BICUBIC, expand=True)

        # Resize using dynamic scale bounds based on background
        original_w, original_h = fg_img.size
        max_scale_w = (bg_img.width * max_size_ratio) / original_w
        max_scale_h = (bg_img.height * max_size_ratio) / original_h
        min_scale_w = (bg_img.width * min_size_ratio) / original_w
        min_scale_h = (bg_img.height * min_size_ratio) / original_h

        scale_min = max(min_scale_w, min_scale_h)
        scale_max = min(max_scale_w, max_scale_h)

        # Clamp to [0.01, 1.0] just in case
        scale_min = max(scale_min, 0.01)
        scale_max = min(scale_max, 1.0)

        # If bounds are inverted due to very large fg, clamp both to min_scale
        if scale_max < scale_min:
            scale_max = scale_min

        scale = random.uniform(scale_min, scale_max)

        new_w = int(original_w * scale)
        new_h = int(original_h * scale)

        fg_img = fg_img.resize((new_w, new_h), resample=Image.BICUBIC)

        fg_img = ImageEnhance.Brightness(
            fg_img).enhance(random.uniform(0.6, 1.4))
        fg_img = ImageEnhance.Contrast(
            fg_img).enhance(random.uniform(0.8, 1.2))
        fg_img = ImageEnhance.Color(fg_img).enhance(random.uniform(0.6, 1.4))
        fg_img = fg_img.filter(ImageFilter.GaussianBlur(
            radius=random.uniform(0.0, 0.8)))

        img[1] = fg_img

        max_x = bg_img.width - fg_img.width
        max_y = bg_img.height - fg_img.height

        for attempt in range(50):
            x = random.randint(0, max_x)
            y = random.randint(0, max_y)

            fg_mask = np.array(fg_img.split()[-1]) > 0

            # Make sure dimensions align (especially after rotation/resize)
            mask_h, mask_w = fg_mask.shape
            if y + mask_h > occupied_mask.shape[0] or x + mask_w > occupied_mask.shape[1]:
                continue  # skip if out of bounds (shouldn’t usually happen)

            occ_crop = occupied_mask[y:y + mask_h, x:x + mask_w]
            fg_mask_area = np.sum(fg_mask)
            occ_crop_area = np.sum(occ_crop)
            overlap = np.sum(np.logical_and(fg_mask, occ_crop))

            allowed_overlap_fg = max_overlap_pct / 100 * fg_mask_area
            allowed_overlap_occ = max_overlap_pct / 100 * \
                occ_crop_area if occ_crop_area > 0 else 0

            # Accept placement only if it doesn't cover too much of either
            if overlap <= allowed_overlap_fg and overlap <= allowed_overlap_occ:
                occupied_mask[y:y + fg_img.height, x:x +
                              fg_img.width] = np.logical_or(occ_crop, fg_mask)
                break
        else:
            continue

        seg_img = fg_img
        img_arr = np.array(seg_img)
        mask = img_arr[:, :, 3] != 0

        segmentation = []
        for i in range(mask.shape[0]):
            for j in range(mask.shape[1]):
                if mask[i, j]:
                    segmentation.append(j + x)
                    segmentation.append(i + y)
        segmentation = [segmentation]

        area = abs(sum(
            segmentation[0][2 * i] * segmentation[0][(2 * i + 3) % len(segmentation[0])] -
            segmentation[0][(2 * i + 2) % len(segmentation[0])
                            ] * segmentation[0][2 * i + 1]
            for i in range(len(segmentation[0]) // 2)
        )) / 2

        bg_img.paste(fg_img, (x, y), fg_img)

        x1, y1 = x, y
        x2 = x + fg_img.width
        y2 = y + fg_img.height

        if x2 <= x1 or y2 <= y1:
            continue

        x_center_ann = ((x1 + x2) / 2) / bg_img.width
        y_center_ann = ((y1 + y2) / 2) / bg_img.height
        width_ann = (x2 - x1) / bg_img.width
        height_ann = (y2 - y1) / bg_img.height

        if not (0 <= x_center_ann <= 1 and 0 <= y_center_ann <= 1 and 0 <= width_ann <= 1 and 0 <= height_ann <= 1):
            continue

        # Create segmentation labels
        normalize_seg = []
        for i in range(len(segmentation[0])):
            if i % 2 == 0:
                val = segmentation[0][i] / bg_img.width
            else:
                val = segmentation[0][i] / bg_img.height
            normalize_seg.append(str(val))
        with open(f'{trainfolder}labels/{img_id}.txt', 'a') as f:
            f.write(f"{annotations_ID[img[0]]} {' '.join(normalize_seg)}\n")

      # Only bbox labels
       # with open(f'{trainfolder}labels/{img_id}.txt', 'a') as f:
        #    f.write(f"{annotations_ID[img[0]]} {x_center_ann} {y_center_ann} {width_ann} {height_ann}\n")

        annotations2.append({
            "segmentation": segmentation, "area": area, "iscrowd": 0,
            "image_id": img_id, "bbox": [x, y, fg_img.width, fg_img.height],
            "category_id": annotations_ID[img[0]], "id": anno_id

        })
        annotations.append({
            "id": anno_id, "image_id": img_id, "category_id": annotations_ID[img[0]],
            "bbox": [x, y, fg_img.width, fg_img.height],
            "segmentation": [], "area": fg_img.height * fg_img.width, "iscrowd": 0
        })
        annot_csv.append([
            "TRAIN", output_folder +
            str(img_id)+".jpg", img[0], x/bg_img.width, y/bg_img.height,
            "", "", (x+fg_img.width) /
            bg_img.width, (y+fg_img.height)/bg_img.height
        ])

        anno_id += 1

    bg_img = bg_img.convert("RGB")
    bg_img.save(f"{trainfolder}images/" + str(img_id) + ".jpg", quality=100)
    images.append({"id": img_id, "file_name": str(img_id) + ".jpg",
                  "height": bg_img.height, "width": bg_img.width})
    img_id += 1
    print(f"Imagen {img_id} generada correctamente con {num_objects} objetos.")

## Create COCO


In [ ]:
# Create coco json
import json

categories = [{"id": cid, "name": name, "supercategory": "none"}
              for name, cid in annotations_ID.items()]

coco_output = {
    "info": {
        "description": "Image Dataset",
        "version": "1.0",
        "year": 2025,
        "contributor": "",
        "date_created": "2025-07-03"
    },
    "licenses": [],
    "images": images,
    "annotations": annotations2,
    "categories": categories
}

with open(f"{output_folder}annotations_coco.json", "w") as jsonfile:
    json.dump(coco_output, jsonfile, indent=4)

## Check generated images

This cell shows n random images from the dataset and draws the bounding boxes from the labels.


In [ ]:
from PIL import Image
from IPython.display import display, Javascript, clear_output
import ipywidgets as widgets
import random
# from PIL import Image, ImageDraw, ImageEnhance, ImageFilter, ImageFont, UnidentifiedImageError
from PIL import ImageDraw
import matplotlib.pyplot as plt
import os
# === CONFIG ===
images_dir = workdir + "ds_final/train/images"
labels_dir = workdir + "ds_final/train/labels"
num_samples = 15
# workdir="/content/"
# Get all image files
image_files = [f for f in os.listdir(
    images_dir) if f.endswith(('.jpg', '.png'))]
sampled_files = random.sample(image_files, min(num_samples, len(image_files)))

for image_file in sampled_files:
    image_path = os.path.join(images_dir, image_file)
    label_file = os.path.splitext(image_file)[0] + ".txt"
    label_path = os.path.join(labels_dir, label_file)

    # Load image
    image = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(image)
    img_width, img_height = image.size

    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            labels = f.readlines()

        for label in labels:
            cls, x_center, y_center, w, h = map(float, label.strip().split())
            x_center *= img_width
            y_center *= img_height
            w *= img_width
            h *= img_height
            x0 = x_center - w / 2
            y0 = y_center - h / 2
            x1 = x_center + w / 2
            y1 = y_center + h / 2
            draw.rectangle([x0, y0, x1, y1], outline="red", width=5)
            draw.text((x0, y0), str(int(cls)), fill="red")

    # Show image with bounding boxes
    plt.figure(figsize=(6, 6))
    plt.imshow(image)
    plt.title(image_file)
    plt.axis("off")
    plt.show()

## Split images

Take a percentage of the images and move them to test and validation directories.


In [ ]:
import os
import shutil
import random

validation = 0.1
test = 0.1

# Assumes train has 100% of data
output_folder = workdir + "ds_final/"
trainfolder = output_folder + "train/"
trainfolderimgs = trainfolder + "images/"
trainfolderlabels = trainfolder + "labels/"
testfolder = output_folder + "test/"
testfolderimgs = testfolder + "images/"
testfolderlabels = testfolder + "labels/"
validfolder = output_folder + "valid/"
validfolderimgs = validfolder + "images/"
validfolderlabels = validfolder + "labels/"

fullSize = len(os.listdir(trainfolderimgs))
validSize = int(fullSize * validation)
testSize = int(fullSize * test)

for i in range(validSize):
    filelist = os.listdir(trainfolderimgs)
    # randomize file list, to not pick files in order
    random.shuffle(filelist)
    filetomove = filelist[i]
    # take out .jpg, .png, etc
    filetomovename = filetomove[:-4]
    # move images
    shutil.move(f"{trainfolderimgs}{filetomove}",
                f"{validfolderimgs}{filetomove}")
    # move labels
    shutil.move(f"{trainfolderlabels}{filetomovename}.txt",
                f"{validfolderlabels}{filetomovename}.txt")
for i in range(testSize):
    filelist = os.listdir(trainfolderimgs)
    # randomize file list, to not pick files in order
    random.shuffle(filelist)
    filetomove = os.listdir(trainfolderimgs)[i]
    # take out .jpg, .png, etc
    filetomovename = filetomove[:-4]
    # move images
    shutil.move(f"{trainfolderimgs}{filetomove}",
                f"{testfolderimgs}{filetomove}")
    # move labels
    shutil.move(f"{trainfolderlabels}{filetomovename}.txt",
                f"{testfolderlabels}{filetomovename}.txt")

# Validation
print(f"Train size is now: {len(os.listdir(trainfolderimgs))}")
print(f"Validation size is now: {len(os.listdir(validfolderimgs))}")
print(f"Test size is now: {len(os.listdir(testfolderimgs))}")

## Create data.yaml


In [ ]:
# Create data.yaml
data = dict(
    train=os.path.join(output_folder, "train"),
    val=os.path.join(output_folder, "valid"),
    test=os.path.join(output_folder, "test"),
    nc=len(annotations_ID),
    names=list(annotations_ID.keys())
)

with open(f'{output_folder}data.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

## Train a yolo model with the dataset


Clean invalid labels and then train yolo.


In [ ]:
import os
# os.environ["MKL_THREADING_LAYER"] = "GNU" # for some systems
from ultralytics import YOLO

os.chdir(workdir)

# Load a YOLOv8 model (you can also use 'yolov8n.yaml', 'yolov8s.yaml', etc.)
model = YOLO("yolo11m.yaml")  # "yolo11m.yaml"

# Train the model
model.train(
    data=workdir + "ds_final/data.yaml",
    epochs=20,  # change to 200
    imgsz=128,
    batch=32,  # 32, 64, 128 or -1
    # degrees=15,
    # translate=0.1,
    # shear=5,
    # scale=0.5,
    # perspective=0.001,
    # flipud=0.3,
    # fliplr=0.5,
    # mosaic=1.0,
    # mixup=0.2,
    # copy_paste=0.1
    patience=25,
    save_period=10,
    cache=True,
    amp=True,
    device='cpu'
)

In [ ]:
# Test the model
from ultralytics import YOLO
import os

os.chdir(workdir)

# Load the best trained model
model = YOLO(workdir + "runs/detect/train15/weights/best.pt")

# Run inference on test images
results = model.predict(
    source=workdir + "ds_final/test/images", save=True, conf=0.5)